# Linking flow cytometry data against `CellMarker`

In [ ]:
!lamin delete test-flow
!lamin init --storage ./test-flow --schema bionty

Let us now consider a flow cytometry example dataset:

In [ ]:
import lamindb as ln
import lnschema_bionty as lb
import readfcs

ln.track()

In [ ]:
filepath = ln.dev.datasets.file_fcs()

In [ ]:
filepath

## Read a FCS file

We use [readfcs](https://lamin.ai/docs/readfcs) to read fcs file into memory.

In [ ]:
adata = readfcs.read(filepath)

In [ ]:
adata.var

## Parse features

We'll use the `CellMarker`-ontology based reference to link features:

```{tip}

By default, human cell markers are used. You can configure a different species via:

```python

ln.Featureset.from_iterable(..., species="mouse")
```

In [ ]:
featureset = ln.Featureset.from_iterable(
    iterable=adata.var["marker"], field=lb.CellMarker.name
)

In [ ]:
ln.save(featureset);

In [ ]:
featureset

In [ ]:
featureset.cell_markers.values_list("name", flat=True)

## Track data with features (cell markers)

```{seealso}

Basic queries: {doc}`/guide/select`

```

In [ ]:
file = ln.File(filepath);

In [ ]:
ln.save(file)

In [ ]:
file.featuresets.add(featureset)

## Querying data by features

We can now query datasets by cell markers:

In [ ]:
ln.File.select(featuresets__cell_markers__gene_symbol="CD14").df()

In [ ]:
# throw error if there is more or less
ln.File.select(featuresets__cell_markers__gene_symbol="CD14").one()
# clean up test instance
!lamin delete test-flow
!rm -r test-flow